<a href="https://colab.research.google.com/github/jaden0305/proj_dbdt/blob/feature_js/team_mission/practices/EDA_of_Chipotle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive

drive.mount('/content/drive/')

# read dataset
chipo = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep='\t')
chipo

Mounted at /content/drive/


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


In [ ]:
# save dataset
chipo.to_csv('~/chipo.tsv', sep='\t')

In [ ]:
# see the first 10 lines
chipo.head(n=10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [ ]:
# What is the number of observations in the dataset?
# What is the number of columns in the dataset?
chipo.shape

(4622, 5)

In [ ]:
# Print the name of all the columns.
# choice_description만 칼럼갯수가 다름
chipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [ ]:
# Which was the most-ordered item?
# For the most-ordered item, how many items were ordered?
chipo_agg = chipo[["item_name", "quantity"]].groupby("item_name").sum().sort_values("quantity", ascending=False)
chipo_agg.head()

,quantity
item_name,
Chicken Bowl,761
Chicken Burrito,591
Chips and Guacamole,506
Steak Burrito,386
Canned Soft Drink,351


In [ ]:
# What was the most ordered item in the choice_description column?
# remove na
chipo_choice_desc = chipo[["choice_description", "quantity"]].dropna()
# remove bracket and split item lists by comma
chipo_choice_desc["choice_items"] = chipo_choice_desc.apply(
    lambda row: row["choice_description"].replace('[','').replace(']','').replace(', ', ',').split(','),
    axis=1
)

# aggregate choice items
choice_items = dict()
def aggregate_items(items:list, qty):
  for item in items:
    choice_items[item] = choice_items.get(item,0) + qty

chipo_choice_desc.apply(
    lambda row: aggregate_items(row["choice_items"], row["quantity"]),
    axis=1
)

sorted_choice_items = sorted(choice_items.items(), key = lambda item: item[1], reverse = True)
sorted_choice_items[0]

('Rice', 2497)

In [ ]:
# How many items were orderd in total?
chipo.quantity.sum()

4972

In [ ]:
# Turn the item price into a float.
chipo["item_price"] = chipo.apply(
    lambda row: float(row["item_price"][1:-1]),
    axis=1
)

chipo.item_price.dtype

dtype('float64')

In [ ]:
# How much was the revenue for the period in the dataset?
revenue = sum(chipo.item_price * chipo.quantity)
print(f"Revenue : ${revenue:.2f}")

Revenue : $39237.02


In [ ]:
# How many orders were made in the period?
chipo.order_id.nunique()

1834

In [ ]:
# What is the average revenue amount per order?
chipo["revenue"] = chipo.apply(
    lambda row: row["item_price"] * row["quantity"],
    axis=1
)

revenue_by_order = chipo.groupby("order_id")["revenue"].sum().reset_index()
revenue_by_order.revenue.mean()

21.39423118865867

In [ ]:
# How many different items are sold?
chipo.item_name.nunique()

50